In [1]:
import pandas as pd
from pprint import pprint 
from tqdm import tqdm

In [10]:
import os
rootdir='.\\'
paths=list()
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = subdir + os.sep + file

        if filepath.endswith(".conll") and filepath.startswith(".\\iter"):
            paths.append(filepath)

        # if filepath.endswith(".conll"):
        #     paths.append(filepath)
pprint(paths)

['.\\iter1\\project-Iterasi 1_#1-300.conll',
 '.\\iter1\\project-Iterasi 1_#1400-1900.conll',
 '.\\iter1\\project-Iterasi 1_#1900-2500.conll',
 '.\\iter1\\project-Iterasi 1_#2500-3000.conll',
 '.\\iter1\\project-Iterasi 1_#300-600.conll',
 '.\\iter1\\project-Iterasi 1_#600-900.conll',
 '.\\iter1\\project-Iterasi 1_#900-1400.conll',
 '.\\iter2\\project-Iterasi 2_#3000-3300-at-2021-06-14-16-41-8f7930b8.conll',
 '.\\iter2\\project-Iterasi 2_#3300-3600-at-2021-06-14-16-41-6c6edff0.conll',
 '.\\iter2\\project-Iterasi 2_#3600-4000-at-2021-06-14-16-41-6b70c208.conll',
 '.\\iter2\\project-Iterasi 2_#4000-4200-at-2021-06-14-16-40-fde4c9f2.conll',
 '.\\iter2\\project-Iterasi 2_#4200-4300-at-2021-06-14-16-40-831cd072.conll',
 '.\\iter2\\project-Iterasi 2_#4300_4800-at-2021-06-14-16-40-2c3205b6.conll',
 '.\\iter2\\project-Iterasi 2_#4800-5300-at-2021-06-14-16-40-7d427527.conll',
 '.\\iter2\\project-Iterasi 2_#5300-5900-at-2021-06-14-16-39-cfb6c668.conll',
 '.\\iter3\\project-Iterasi 3_#5900-6200-a

In [15]:
result=list()
x='?.!,-~!$%&()*+,-./:;<=>?[\]^_`{|}~'

for path in paths:
    f = open(path,"r", encoding='utf-8') 
    sym=''
    c= 0
    index=0
    for line in f.readlines():
        # try:
        line_split=line.split(" ")
        n_token=len(line_split)
        token=line_split[0]
        # print(n_token)
        ner=line_split[-1].replace("\n","")
        
        # skip unwanted parts of the data
        if len(token.strip())==0 or token=="-DOCSTART-": 
            # print(token)
            continue
        elif token.isdigit() and len(str(token))==19:
            id=token
            index= 0
        else:
           # if (index==0 and token[0]=='"'):# or index==n_token-1 or index==n_token:
            #     print(token,n_token)
            #     continue
            if len(token)>1 and any(c.isalpha() for c in token)  and ((token[0] in '\'"' and token[-1] not in '\'"') or token[0] in x):
                sym=token[0]
                token=token[1:].strip(x)
                result.append({'token':sym, 'ner':'O', 'tweet_id':id, 'index':index})
                index+=1
                n_token+=1

            if len(token)>1 and any(c.isalpha() for c in token) and ((token[0] not in '\'"' and token[-1] in '\'"') or token[-1] in x):
                sym=token[-1]
                token=token[:-1].strip(x)
                result.append({'token':token, 'ner':ner, 'tweet_id':id, 'index':index})
                index+=1
                n_token+=1
                result.append({'token':sym, 'ner':'O', 'tweet_id':id, 'index':index})
            else:
                result.append({'token':token, 'ner':ner, 'tweet_id':id, 'index':index})
            index+=1
        # except IndexError as e:
        #     continue
# pprint(result)

In [16]:
df_result= pd.DataFrame(result)
df_result

,token,ner,tweet_id,index
0,"""",O,1382601382042103808,0
1,Hidup,O,1382601382042103808,1
2,sesedih,O,1382601382042103808,2
3,dan,O,1382601382042103808,3
4,secaper,O,1382601382042103808,4
...,...,...,...,...
184917,Ngabuburit,O,1386301391086309382,1
184918,tadi,O,1386301391086309382,2
184919,sore,O,1386301391086309382,3
184920,https://t.co/13WztrZOAX,O,1386301391086309382,4


In [17]:
df_result.tweet_id.value_counts()

1388364959113891840    138
1385842778286923776    124
1389542952754630659    122
1388541528159625217    120
1389489993039503362    116
                      ... 
1389912171878903808      4
1390676454304739332      4
1383183332440100866      4
1390912076101537795      4
1381033248252252163      2
Name: tweet_id, Length: 7426, dtype: int64

In [34]:
print(df_result[df_result.tweet_id=="1383265898832617475"])

         token    ner             tweet_id  index
41498        "      O  1383265898832617475      0
41499  taeyong  B-PER  1383265898832617475      1
41500    jogja  B-LOC  1383265898832617475      2
41501        "      O  1383265898832617475      3


In [19]:
df_result.loc[df_result.groupby(['tweet_id']).index.idxmin()]

,token,ner,tweet_id,index
48650,"""",O,1380681275594305537,0
86906,"""",O,1380685209075544064,0
17951,"""",O,1380685355897085952,0
80921,"""",O,1380686115007393792,0
123122,"""",O,1380688640611786754,0
...,...,...,...,...
172412,"""",O,1391611459931541506,0
97794,"""",O,1391612015286702084,0
30656,"""",O,1391614165182128130,0
81379,"""",O,1391614604388622337,0


In [20]:
remove_index= list(df_result.groupby(['tweet_id']).index.idxmin())
remove_index.extend(list(df_result.groupby(['tweet_id']).index.idxmax()))
print(len(remove_index))

14852


In [21]:
df_result.drop(remove_index, inplace=True)
df_result.reset_index(drop=True,inplace=True)

In [22]:
df_result["index"]=df_result["index"]-1
df_result

,token,ner,tweet_id,index
0,Hidup,O,1382601382042103808,0
1,sesedih,O,1382601382042103808,1
2,dan,O,1382601382042103808,2
3,secaper,O,1382601382042103808,3
4,apa,O,1382601382042103808,4
...,...,...,...,...
170065,muntah,O,1386004972207153156,5
170066,Ngabuburit,O,1386301391086309382,0
170067,tadi,O,1386301391086309382,1
170068,sore,O,1386301391086309382,2


In [23]:
df_result.ner.value_counts()

O         148104
B-PER       5564
B-PROD      3554
I-PER       3058
B-LOC       2225
B-ORG       2085
I-PROD      1177
I-ORG       1124
I-EV         978
I-LOC        957
B-EV         809
I-WA         271
B-WA         164
Name: ner, dtype: int64

In [40]:
df_result[df_result['tweet_id']==1388823812091748354]

,token,ner,tweet_id,index


In [41]:
df_result.loc[2471]

token            #THRDariShopee
ner                           O
tweet_id    1386408258781335554
index                        17
Name: 2471, dtype: object

In [25]:
df_result.drop(['tweet_id','index'],axis=1)

,token,ner
0,Hidup,O
1,sesedih,O
2,dan,O
3,secaper,O
4,apa,O
...,...,...
170065,muntah,O
170066,Ngabuburit,O
170067,tadi,O
170068,sore,O


In [42]:
df_result.to_csv('annotated_ner_data_new.csv',index=False)